Passe les fichiers en lecture

In [88]:


import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import datetime

file_path_list = ["C:\Esilv\Annee 3\Semestre 6\Python\Valeursfoncieres-2021\Valeursfoncieres-2018.txt",
                  "C:\Esilv\Annee 3\Semestre 6\Python\Valeursfoncieres-2021\Valeursfoncieres-2019.txt",
                  "C:\Esilv\Annee 3\Semestre 6\Python\Valeursfoncieres-2021\Valeursfoncieres-2020.txt",
                  "C:\Esilv\Annee 3\Semestre 6\Python\Valeursfoncieres-2021\Valeursfoncieres-2021.txt",
                  "C:\Esilv\Annee 3\Semestre 6\Python\Valeursfoncieres-2021\Valeursfoncieres-2022.txt"]

for i in range(len(file_path_list)):
    file_path=file_path_list[i]
    if os.access(file_path, os.R_OK):
        print('File has read permission')
    else:
        print('File does not have read permission')
    
    os.chmod(file_path, 0o400)


del file_path, file_path_list

File has read permission
File has read permission
File has read permission
File has read permission
File has read permission


Ouvre l'année 2022 du des fichiers de valeurs foncières

In [89]:
def load_dataframe(annee):
    if(annee == 2018):
        dataframe = pd.read_csv("C:\Esilv\Annee 3\Semestre 6\Python\Valeursfoncieres-2021\Valeursfoncieres-2018.txt", sep="|")
        return dataframe
    if(annee == 2019):
        dataframe = pd.read_csv("C:\Esilv\Annee 3\Semestre 6\Python\Valeursfoncieres-2021\Valeursfoncieres-2019.txt", sep="|")
        return dataframe
    if(annee == 2020):
        dataframe = pd.read_csv("C:\Esilv\Annee 3\Semestre 6\Python\Valeursfoncieres-2021\Valeursfoncieres-2020.txt", sep="|")
        return dataframe
    if(annee == 2021):
        dataframe = pd.read_csv("C:\Esilv\Annee 3\Semestre 6\Python\Valeursfoncieres-2021\Valeursfoncieres-2021.txt", sep="|")
        return dataframe     
    if(annee == 2022):
        dataframe = pd.read_csv("C:\Esilv\Annee 3\Semestre 6\Python\Valeursfoncieres-2021\Valeursfoncieres-2022.txt", sep="|")
        return dataframe   
#dataframe2019 = pd.read_csv("C:\Esilv\Annee 3\Semestre 6\Python\Valeursfoncieres-2021\Valeursfoncieres-2019.txt", sep="|")
#dataframe2020 = pd.read_csv("C:\Esilv\Annee 3\Semestre 6\Python\Valeursfoncieres-2021\Valeursfoncieres-2020.txt", sep="|")
#dataframe2021 = pd.read_csv("C:\Esilv\Annee 3\Semestre 6\Python\Valeursfoncieres-2021\Valeursfoncieres-2021.txt", sep="|")
#dataframe2022 = pd.read_csv("C:\Esilv\Annee 3\Semestre 6\Python\Valeursfoncieres-2021\Valeursfoncieres-2022.txt", sep="|")


Se débarasse des colonnes inutiles

In [90]:
def colonnes_drop(dataframe):
    dataframe=dataframe.drop(
                    ["Identifiant de document",
                    "Reference document",
                    "1 Articles CGI",
                    "2 Articles CGI",
                    "3 Articles CGI",
                    "4 Articles CGI",
                    "5 Articles CGI",
                    "No disposition",
                    "Code commune",
                    "Prefixe de section",
                    "Section",
                    "No plan",
                    "No Volume",
                    "1er lot",
                    "2eme lot",
                    "3eme lot",
                    "4eme lot",
                    "5eme lot",
                    "Identifiant local",
                    "Code voie"
                    ],axis=1)
    return dataframe

Supprime les valeurs non renseignées dans les colonnes essentielles. On a besoin de la date de la nature et de la valeur foncière.

In [91]:


def nettoyage_valeurs_essentielles(dataframe):
    list_colonnes_utiles=["Date mutation","Nature mutation","Valeur fonciere"]
    indexes_to_drop=[]
    for nom in list_colonnes_utiles: 
        indexes_to_drop += list(np.where(pd.isnull(dataframe[nom]))[0])
    indexes_to_drop = list(set(indexes_to_drop))  # On supprime les doublons
    dataframe = dataframe.drop(indexes_to_drop)
    dataframe = dataframe.reset_index(drop=True)
    del nom
    del indexes_to_drop
    del list_colonnes_utiles
    return dataframe




On va maintenant rajouter les colonnes qui nous manquent à cette dataframe

In [92]:
fr_departments = {
    "Auvergne-Rhône-Alpes": ["1", "3", "7", "15", "26", "38", "42", "43", "63", "69", "73", "74"],
    "Bourgogne-Franche-Comté": ["21", "25", "39", "58", "70", "71", "89", "90"],
    "Bretagne": ["22", "29", "35", "56"],
    "Centre-Val de Loire": ["18", "28", "36", "37", "41", "45"],
    "Corse": ["2A", "2B"],
    "Grand Est": ["8", "10", "51", "52", "54", "55", "57", "67", "68", "88"],
    "Hauts-de-France": ["2", "59", "60", "62", "80"],
    "Île-de-France": ["75", "77", "78", "91", "92", "93", "94", "95"],
    "Normandie": ["14", "27", "50", "61", "76"],
    "Nouvelle-Aquitaine": ["16", "17", "19", "23", "24", "33", "40", "47", "64", "79", "86", "87"],
    "Occitanie": ["09", "11", "12", "30", "31", "32", "34", "46", "48", "65", "66", "81", "82"],
    "Pays de la Loire": ["44", "49", "53", "72", "85"],
    "Provence-Alpes-Côte d'Azur": ["4", "5", "6", "13", "83", "84"]
}
#print(fr_departments)
def rajout_region(dataframe,fr_departments):
    dataframe["Region"] = np.nan
    for i in range(len(dataframe["Code departement"])):
        valeurdepartement = dataframe["Code departement"][i]
        for j in fr_departments:
            if(str(valeurdepartement) in fr_departments[j]):
                dataframe["Region"][i] = j
                break
    print(dataframe["Region"])
    del valeurdepartement,j,i
    return dataframe
#del fr_departments

On cherche désormais à déterminer le prix au mètre carré et la surface. On découpe la dataframe entre les entrées comptant plusieurs lots et celles comptant un unique lot.

In [93]:



#Nettoyage des valeurs non renseignées
def nettoyage_split_lot(dataframe):
    dataframe_multi_lots = dataframe[dataframe['Nombre de lots'] >= 1]
    dataframe_uni_lots = dataframe[dataframe['Nombre de lots'] == 0]
    dataframe_multi_lots = dataframe_multi_lots.reset_index(drop=True)
    dataframe_uni_lots = dataframe_uni_lots.reset_index(drop=True)
    list_index_to_drop=[]
    for i in range(len(dataframe_multi_lots)):
        if(pd.isnull(dataframe_multi_lots["Surface Carrez du 1er lot"][i])
        and pd.isnull(dataframe_multi_lots["Surface Carrez du 2eme lot"][i])
        and pd.isnull(dataframe_multi_lots["Surface Carrez du 3eme lot"][i])
        and pd.isnull(dataframe_multi_lots["Surface Carrez du 4eme lot"][i])
        and pd.isnull(dataframe_multi_lots["Surface Carrez du 5eme lot"][i])):
            list_index_to_drop.append(i)
    dataframe_multi_lots = dataframe_multi_lots.drop(list_index_to_drop)

    list_index_to_drop=[]
    for i in range(len(dataframe_uni_lots)):
        if(pd.isnull(dataframe_uni_lots["Surface reelle bati"][i])
        and pd.isnull(dataframe_uni_lots["Surface terrain"][i])):
            list_index_to_drop.append(i)
    dataframe_uni_lots = dataframe_uni_lots.drop(list_index_to_drop)

    #Après nettoyage il faut reformater les dataframes
    dataframe_multi_lots = dataframe_multi_lots.reset_index(drop=True)
    dataframe_uni_lots = dataframe_uni_lots.reset_index(drop=True)

    surface_multi=[]
    list_colonne_multi=["Surface Carrez du 1er lot",
                        "Surface Carrez du 2eme lot",
                        "Surface Carrez du 3eme lot",
                        "Surface Carrez du 4eme lot",
                        "Surface Carrez du 5eme lot"]
    for i in range(len(dataframe_multi_lots)):
        surface_totale=0
        for colonne in list_colonne_multi:
            nombre_str = dataframe_multi_lots[colonne][i]
            if(isinstance(nombre_str,str)):
                nombre_str=nombre_str.replace(",",".")
            if pd.isna(nombre_str):
                surface_totale += 0.0  # Ajouter une valeur nulle pour les valeurs manquantes
            else:
                surface_totale += float(nombre_str)
        surface_multi.append(surface_totale)

    surface_uni=[]
    list_colonne_uni=["Surface reelle bati",
                    "Surface terrain"]
    for i in range(len(dataframe_uni_lots)):
        surface_totale=0
        for colonne in list_colonne_uni:
            nombre_str = dataframe_uni_lots[colonne][i]
            if pd.isna(nombre_str):
                surface_totale += 0.0  # Ajouter une valeur nulle pour les valeurs manquantes
            else:
                surface_totale += float(nombre_str)
        surface_uni.append(surface_totale) 
    
    #Rajout des colonnes aux dataframe
    dataframe_multi_lots["Surface"]=surface_multi
    dataframe_uni_lots["Surface"]=surface_uni
    
    frames = [dataframe_multi_lots,dataframe_uni_lots]
    dataframe=pd.concat(frames)
    dataframe = dataframe.reset_index(drop=True)

    for i in range(len(dataframe)):
        valeur=(dataframe.loc[i, "Valeur fonciere"])
        if(isinstance(valeur,str)):
            valeur=valeur.replace(",",".")
            dataframe.at[i,'Valeur fonciere']=valeur

    dataframe["Valeur fonciere"]=dataframe["Valeur fonciere"].astype(float)

    #Calcul du prix au metre carre
    dataframe["Prix metre carre"]= dataframe["Valeur fonciere"] / dataframe["Surface"]
    return dataframe

## Conversion des données

In [ ]:
def conversion_dates(dataframe):
    dataframe['Date mutation'] = pd.to_datetime(dataframe['Date mutation'])

## Exécution du traitement de données :

In [94]:
choix_annee_1 = int(input("Quelle année souhaitez vous consulter ?"))
dataframe_lecture = load_dataframe(choix_annee_1)
dataframe_lecture = colonnes_drop(dataframe_lecture)
dataframe_lecture = nettoyage_valeurs_essentielles(dataframe_lecture)
dataframe_lecture = rajout_region(dataframe_lecture,fr_departments)
#split_lot(dataframe_lecture)
dataframe_lecture = nettoyage_split_lot(dataframe_lecture)

C:\Users\user\AppData\Local\Temp\ipykernel_24196\1479475257.py:3: DtypeWarning: Columns (14,18,23,24,26,28,29,31,32,33,41) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframe = pd.read_csv("C:\Esilv\Annee 3\Semestre 6\Python\Valeursfoncieres-2021\Valeursfoncieres-2018.txt", sep="|")
C:\Users\user\AppData\Local\Temp\ipykernel_24196\4117970547.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe["Region"][i] = j


0          Auvergne-Rhône-Alpes
1          Auvergne-Rhône-Alpes
2          Auvergne-Rhône-Alpes
3          Auvergne-Rhône-Alpes
4          Auvergne-Rhône-Alpes
                   ...         
3303280           Île-de-France
3303281           Île-de-France
3303282           Île-de-France
3303283           Île-de-France
3303284           Île-de-France
Name: Region, Length: 3303285, dtype: object


## Fonctions de filtrage dataframe


In [ ]:
def filtre_dataframe_par_date(dataframe, date_debut, date_fin):
    mask = (dataframe["Date mutation"] >= date_debut) & (dataframe["Date mutation"] <= date_fin)
    dataframe_filtre = dataframe.loc[mask]
    return dataframe_filtre

def multi_filtre_dataframe(dataframe, colonne, values): #Permet de filtrer la dataframe selon une liste de valeurs d'une colonne
    if isinstance(values, list):
        # Si la valeur est une liste, filtrer la DataFrame avec la méthode isin
        df_filtree = dataframe[dataframe[colonne].isin(values)]
    else:
        # Si la valeur est une valeur unique, filtrer la DataFrame avec l'opérateur ==
        df_filtree = dataframe[dataframe[colonne] == values]
    return df_filtree

def prix_selon_param(dataframe, colonne, value, ascend): #Filtre et renvoie par ordre croissant ou décroissant la dataframe
    df_filtre = multi_filtre_dataframe(dataframe,colonne,value)
    if(ascend == True):
        df_filtre = df_filtre.sort_values(by=['Prix metre carre'],ascending=True)
    else:
        df_filtre = df_filtre.sort_values(by=['Prix metre carre'],ascending=False)
        
    #return df_filtre[['Prix metre carre','Commune','Code departement','Region','Type local','Nature culture', colonne]]
    return df_filtre

def tranches_de_param(dataframe,colonne,valeur_sup,valeur_inf=None): #Générique de date, renvoie la dataframe sous une tranche d'infos
    if(valeur_inf != None):
        mask = (dataframe[colonne]>= valeur_inf) & (dataframe[colonne]<= valeur_sup)
    else:
        mask = (dataframe[colonne]>= valeur_inf)
    dataframe_filtre = dataframe.loc[mask]
    return dataframe_filtre

def affichage_colonnes(dataframe, colonnes_a_virer): #Demande à l'utilisateur quelles colonnes il souhaite afficher
    dataframe_retour = dataframe.drop(colonnes_a_virer,axis=1)
    return dataframe_retour

def occurrences_data(dataframe,colonne,valeur): #Combien de fois apparaît une valeur spécifiée
    return dataframe[colonne].value_counts()[valeur]

## Fonctions de graphiques

Quantité de vente pour une zone donnée

In [ ]:
def ventes_par_region(dataframe,colonne):
    ventes_par_region = dataframe[colonne].value_counts()
    fig, ax = plt.subplots()
    ventes_par_region.plot(kind='bar', ax=ax)
    ax.set_title('Nombre de ventes par '+colonne)
    ax.set_xlabel(colonne)
    ax.set_ylabel('Nombre de ventes')
    plt.show()


def ventes_par_region_camember(dataframe,colonne):
    ventes_par_region = dataframe[colonne].value_counts()
    fig, ax = plt.subplots()
    ax.pie(ventes_par_region, labels=ventes_par_region.index, autopct='%1.1f%%')
    ax.set_title('Répartition des ventes par '+colonne)
    plt.show()

    

Prix moyen par région donnée

## Options

filtre par date, 
filtre par valeur d'une région / département / commune, 
filtre par maximum et minimum de prix, 
filtre par maximum et minimum de surface, 
Ordonnancement des ventes par prix croisant ou décroissant du m², 

## A faire
Analyse des transactions immobilières par type de bien
Analyse des transactions immobilières par type de bien
Analyse des prix de vente par nature de mutation
Analyse des prix de vente par surface réelle bâtie
Analyse des prix de vente par nombre de pièces principales